###  Test of python compatibility

We want this notebook to run using python3 both in the driver and in the workers.


## Reading data and creating dataframe

In [1]:
%%time
%pwd
#import findspark
#findspark.init()

from pyspark import SparkContext
libfilename='/mnt/workspace/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Section2-PCA/PCA/data_preparation/lib/numpy_pack.py'
#libfilename='./lib/numpy_pack.py'
sc = SparkContext(pyFiles=[libfilename])

from pyspark import SparkContext
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

CPU times: user 164 ms, sys: 16 ms, total: 180 ms
Wall time: 12.3 s


In [2]:
%%time
file_index='SSSSBBBB'
data_dir='../../../Data/Weather'

!ls -l $data_dir
filebase='US_Weather_%s'%file_index
!rm -rf $data_dir/$filebase*

c_filename=filebase+'.csv.gz'
u_filename=filebase+'.csv'

command="curl https://mas-dse-open.s3.amazonaws.com/Weather/small/%s > %s/%s"%(c_filename,data_dir,c_filename)
print(command)
!$command
!ls -lh $data_dir/$c_filename

total 56392
drwxrwxr-x 2 hadoop hadoop    36864 Mar  6 20:44 decon_SSSSBBBB_SNWD.parquet
-rw-rw-r-- 1 hadoop hadoop 25674261 Mar  6 20:44 STAT_SSSSBBBB.pickle
-rw-rw-r-- 1 hadoop hadoop 12456904 Mar  6 20:44 US_Weather_BBSSBBSS.csv
-rw-rw-r-- 1 hadoop hadoop  3430874 Mar  6 20:44 US_Weather_BBSSBBSS.csv.gz
drwxrwxr-x 2 hadoop hadoop     4096 Mar  6 20:44 US_Weather_BBSSBBSS.parquet
-rw-rw-r-- 1 hadoop hadoop 12880638 Mar  6 20:44 US_Weather_SSSSBBBB.csv
-rw-rw-r-- 1 hadoop hadoop  3245918 Mar  6 20:44 US_Weather_SSSSBBBB.csv.gz
curl https://mas-dse-open.s3.amazonaws.com/Weather/small/US_Weather_SSSSBBBB.csv.gz > ../../../Data/Weather/US_Weather_SSSSBBBB.csv.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3169k  100 3169k    0     0  1584k      0  0:00:02  0:00:02 --:--:-- 1403k
-rw-rw-r-- 1 hadoop hadoop 3.1M Mar  7 03:46 ../../../Data/Weather/US_Weather_SSSSBBBB.csv.gz


In [4]:
#unzip
!gunzip -c $data_dir/$c_filename > $data_dir/$u_filename
!ls -l $data_dir

total 56392
drwxrwxr-x 2 hadoop hadoop    36864 Mar  6 20:44 decon_SSSSBBBB_SNWD.parquet
-rw-rw-r-- 1 hadoop hadoop 25674261 Mar  6 20:44 STAT_SSSSBBBB.pickle
-rw-rw-r-- 1 hadoop hadoop 12456904 Mar  6 20:44 US_Weather_BBSSBBSS.csv
-rw-rw-r-- 1 hadoop hadoop  3430874 Mar  6 20:44 US_Weather_BBSSBBSS.csv.gz
drwxrwxr-x 2 hadoop hadoop     4096 Mar  6 20:44 US_Weather_BBSSBBSS.parquet
-rw-rw-r-- 1 hadoop hadoop 12880638 Mar  7 03:47 US_Weather_SSSSBBBB.csv
-rw-rw-r-- 1 hadoop hadoop  3245918 Mar  7 03:46 US_Weather_SSSSBBBB.csv.gz


In [5]:
import pickle
List=pickle.load(open(data_dir+'/'+u_filename,'rb'))

In [6]:
Weather_df=sqlContext.createDataFrame(List)
print(Weather_df.count())
Weather_df.show(2)

12740
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|   2449.1| 36.0333|-105.8167|       TMAX|USC00299113|     6|[40 56 30 54 40 5...|1910.0|SSSSBBBB|
|   2449.1| 36.0333|-105.8167|       TMAX|USC00299113|    21|[80 C9 30 D4 00 4...|1911.0|SSSSBBBB|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 2 rows



In [8]:
#processing a single row

In [7]:
row,=Weather_df.take(1)
V=row.vector
V

import sys
sys.path.append('./lib')
from numpy_pack import unpackArray

import numpy as np
unpackArray(V,data_type=np.float16)

array([100.,  67.,  50., -11., -83., -89., -39.,   0.,  22.,  67.,  50.,
        83.,  17.,  44.,  72.,  28.,  22.,  -6.,  44.,  61.,  78., 100.,
       100.,  nan,  78.,  44.,  39.,  44.,  50.,  61.,  39.,  78.,  17.,
        17.,  nan,  nan,  33.,  44.,  28., -28.,  17.,  17.,  28.,  78.,
       111.,  nan,  44., -72.,   0., -11., -17.,  39.,  78.,  50., 100.,
        72.,  50.,  61.,  56., 122., 150., 150., 156., 189., 183., 178.,
       133., 150.,  94., 128., 133., 156., 100.,  94., 122., 144., 167.,
       172., 167., 172., 183., 161., 167., 150., 122., 150., 133.,  17.,
        44., 106., 128., 150., 200., 117., 144., 100., 122., 189.,  94.,
       139., 150., 156., 106., 111.,  39.,  78.,  39., 167., 189., 211.,
       183., 183., 183., 189., 206., 217., 228., 239., 217., 228., 150.,
        nan, 150., 167., 167., 178., 183., 211., 239., 250., 244., 228.,
       244., 183., 200., 172., 128., 133., 194., 178., 133., 117., 172.,
       228., 183., 217., 239., 283., 278., 267., 26

## Processing all rows in a DataFrame
using UDF

In [10]:
from pyspark.sql.types import ArrayType,FloatType, DoubleType, StringType, IntegerType
from pyspark.sql.functions import udf

def Unpack(V):
    A=[float(x) for x in unpackArray(V,data_type=np.float16)]
    return A
Unpack_udf = udf(Unpack,ArrayType(FloatType(),True))
sqlContext.registerFunction("UnpackArray", Unpack,ArrayType(FloatType()))
sqlContext.registerDataFrameAsTable(Weather_df,'weather')

Weather_df2=Weather_df.withColumn("Array", Unpack_udf(Weather_df.vector)).drop('vector')
Weather_df2.show(2)

+---------+--------+---------+-----------+-----------+------+------+--------+--------------------+
|elevation|latitude|longitude|measurement|    station|undefs|  year|   label|               Array|
+---------+--------+---------+-----------+-----------+------+------+--------+--------------------+
|   2449.1| 36.0333|-105.8167|       TMAX|USC00299113|     6|1910.0|SSSSBBBB|[100.0, 67.0, 50....|
|   2449.1| 36.0333|-105.8167|       TMAX|USC00299113|    21|1911.0|SSSSBBBB|[-11.0, -67.0, 28...|
+---------+--------+---------+-----------+-----------+------+------+--------+--------------------+
only showing top 2 rows

